Model trained on contaxts being original_code. 

In [123]:
from transformers import (
    BartTokenizerFast,
    DataCollatorForSeq2Seq,
    keras_callbacks,
    TFAutoModelForSeq2SeqLM,
)
import tensorflow as tf
from datasets import Dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
from huggingface_hub import notebook_login
from datasets import concatenate_datasets
from question_answering.constants import constants
from question_answering.utils import core_qa_utils, generative_qa_utils
from question_answering.paths import generative_qa_paths
from question_answering.keras_callbacks.time_measure_callback import TimeMeasureCallback

In [124]:
df_train, df_val, df_test = core_qa_utils.load_datasets_from_csv(
    generative_qa_paths.python_dataset_dir
)

df_train = pd.concat([df_train, df_val], ignore_index=True)

train_dataset, test_dataset = core_qa_utils.convert_dataframes_to_datasets(
    [df_train, df_test]
)

In [3]:
model_checkpoint = "facebook/bart-base"
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")

In [4]:
def tokenize_sample(sample, max_tokens=None, padding=False):
    question = sample["questions"].strip()
    context = sample["original_code"].strip()

    return tokenizer(question, context, max_length=max_tokens, padding=padding)

tokenized_train_dataset = train_dataset.map(tokenize_sample)
tokenized_test_dataset = test_dataset.map(tokenize_sample)

print(
    "Max number of tokens in tokenized train dataset: ",
    len(max(tokenized_train_dataset["input_ids"], key=len)),
)
print(
    "Max number of tokens in tokenized test dataset: ",
    len(max(tokenized_test_dataset["input_ids"], key=len)),
)

Map:   0%|          | 0/63080 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1490 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Max number of tokens in tokenized train dataset:  9244
Max number of tokens in tokenized test dataset:  901


In [5]:
max_length = 256


def filter_samples_below_number_of_tokens(dataset, max_tokens: int):
    indices_to_remove = []

    # Find indices of samples where number of tokens exceeds max number of tokens
    for index, sample in enumerate(dataset):
        tokenized_sample = tokenize_sample(sample)
        if len(tokenized_sample["input_ids"]) > max_tokens:
            indices_to_remove.append(index)

    # Keep only samples with number of tokens less or equal than max number of tokens
    dataset_indices = range(len(dataset))
    filtered_dataset = dataset.select(
        index for index in dataset_indices if index not in set(indices_to_remove)
    )

    return filtered_dataset


filtered_train_dataset = filter_samples_below_number_of_tokens(
    train_dataset, max_tokens=max_length
)
filtered_test_dataset = filter_samples_below_number_of_tokens(
    test_dataset, max_tokens=max_length
)

Parameter 'indices'=<generator object filter_samples_below_number_of_tokens.<locals>.<genexpr> at 0x000001C5462C8CF0> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [6]:
print(
    "Number of samples in tokenized train dataset before filtering: ",
    len(train_dataset),
)
print(
    "Number of samples in tokenized test dataset before filtering: ", len(test_dataset)
)

print("\n---------------\n")

print(
    "Number of samples in tokenized train dataset after filtering: ",
    len(filtered_train_dataset),
)
print(
    "Number of samples in tokenized test dataset after filtering: ",
    len(filtered_test_dataset),
)

Number of samples in tokenized train dataset before filtering:  63080
Number of samples in tokenized test dataset before filtering:  3500

---------------

Number of samples in tokenized train dataset after filtering:  61758
Number of samples in tokenized test dataset after filtering:  3430


In [7]:
def preprocess_dataset(dataset):
    questions = [q.strip() for q in dataset["questions"]]
    contexts = [c.strip() for c in dataset["original_code"]]
    answers = [c.strip() for c in dataset["answers"]]

    inputs = tokenizer(
        questions,
        contexts,
        text_target=answers,
        max_length=max_length,
        truncation=True
    )

    return inputs

In [8]:
tokenized_train_dataset = filtered_train_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_train_dataset.column_names,
)
tokenized_test_dataset = filtered_test_dataset.map(
    preprocess_dataset,
    batched=True,
    remove_columns=filtered_test_dataset.column_names,
)

Map:   0%|          | 0/61758 [00:00<?, ? examples/s]

Map:   0%|          | 0/3430 [00:00<?, ? examples/s]

In [3]:
# Model parameters
training_number = 1

model_name = "python-bart-uncased"
full_model_name = f"{model_name}-{training_number}"

# Checkpoints
checkpoint_filename_template = constants.checkpoint_filename_template
checkpoints_path = (
    generative_qa_paths.training_checkpoints_dir
    / full_model_name
    / checkpoint_filename_template
)

# Hub
hub_path = generative_qa_paths.hub_models_location / full_model_name

# Saved models
saved_models_path = generative_qa_paths.saved_models_dir / full_model_name

# Evaluation
model_evaluation_dir = generative_qa_paths.model_evaluation_dir / full_model_name

# Hyperparameters
batch_size = 8
train_epochs = 3

In [5]:
# Load model for fine-tuning
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [6]:
# Dataset preparation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

tf_train_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_train_dataset,
    collator=data_collator,
    batch_size=batch_size,
    shuffle=True
)

tf_test_dataset = core_qa_utils.prepare_tf_dataset(
    model=model,
    hf_dataset=tokenized_test_dataset,
    collator=data_collator,
    batch_size=batch_size,
)

NameError: name 'tokenized_train_dataset' is not defined

In [ ]:
# Callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoints_path, verbose=1, save_weights_only=True
)
# push_to_hub = keras_callbacks.PushToHubCallback(
#     output_dir=full_model_name, tokenizer=tokenizer
# )

time_measure_cb = TimeMeasureCallback()

callbacks = [
    checkpoint_cb,
    # push_to_hub,
    time_measure_cb
]

In [ ]:
# Compile
num_train_steps = len(tf_train_dataset) * train_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Compile
model.compile(optimizer=optimizer)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070, compute capability 8.9


In [ ]:
model.summary()

Model: "tf_bart_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  139420416 
                                                                 
 final_logits_bias (BiasLaye  multiple                 50265     
 r)                                                              
                                                                 
Total params: 139,470,681
Trainable params: 139,420,416
Non-trainable params: 50,265
_________________________________________________________________


In [10]:
model

In [15]:
# Fit the model on the new data
history = model.fit(
    tf_train_dataset,
    epochs=train_epochs,
    callbacks=callbacks,
)

Epoch 1/3
7719/7719 [==============================] - ETA: 0s - loss: 2.7848
Epoch 1: saving model to e:\STUDIA\IPS\question-answering\generative-qa\training-checkpoints\python-bart-uncased-1\cp-01.ckpt
7719/7719 [==============================] - 982s 124ms/step - loss: 2.7848
Epoch 2/3
7719/7719 [==============================] - ETA: 0s - loss: 2.0790
Epoch 2: saving model to e:\STUDIA\IPS\question-answering\generative-qa\training-checkpoints\python-bart-uncased-1\cp-02.ckpt
7719/7719 [==============================] - 953s 123ms/step - loss: 2.0790
Epoch 3/3
7719/7719 [==============================] - ETA: 0s - loss: 1.6685
Epoch 3: saving model to e:\STUDIA\IPS\question-answering\generative-qa\training-checkpoints\python-bart-uncased-1\cp-03.ckpt
7719/7719 [==============================] - 951s 123ms/step - loss: 1.6685


In [16]:
# Get best version of the model
best_model, best_epoch = core_qa_utils.get_best_model_from_checkpoints(
    model, history, model_name=full_model_name, metric="loss", remove_checkpoints=True, model_type="generative"
)

In [17]:
# Save best model's weights
generative_qa_utils.save_model(best_model, model_name=full_model_name)

In [15]:
loaded_weights_model = generative_qa_utils.load_weights_into_model(
    model=model, 
    model_name=full_model_name
)

In [19]:
# Get predictions from the best model
loaded_model_evaluation = loaded_weights_model.evaluate(tf_test_dataset)

429/429 [==============================] - 29s 53ms/step - loss: 2.6181


In [20]:
parts_number = 100

test_dataset_pandas = tokenized_test_dataset.to_pandas()
test_dataset_array = np.array_split(test_dataset_pandas, parts_number)

test_datasets_list = []

for row in tqdm(test_dataset_array):
    test_datasets_list.append(core_qa_utils.prepare_tf_dataset(
        model=model,
        hf_dataset=Dataset.from_pandas(row),
        collator=data_collator,
        batch_size=batch_size,
))

c:\Users\Artur\.conda\envs\question_answering\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 100/100 [00:16<00:00,  5.98it/s]


In [21]:
questions_and_answers_df = pd.DataFrame()
index_to_start_from = 0
predictions_list = []
labels_list = []
question_contexts_list = []

i = 0
for dataset in tqdm(test_datasets_list):
    if i < index_to_start_from:
        i = i + 1
        continue
    for batch, labels in tqdm(dataset):
        predictions = generative_qa_utils.generate_predictions(model, batch, max_length)
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_predictions = [prediction.strip() for prediction in decoded_predictions]
        decoded_labels = [label.strip() for label in decoded_labels]
        predictions_list.extend(decoded_predictions)
        labels_list.extend(decoded_labels)
        question_contexts_list.extend(tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True))

        data = {
            'question_contexts': tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True),
            'labels': decoded_labels,
            'predictions': decoded_predictions
        }
        questions_and_answers_df = pd.concat([questions_and_answers_df, pd.DataFrame(data)], ignore_index=True)
    i = i + 1

100%|██████████| 100/100 [47:49<00:00, 28.69s/it]


In [36]:
dataset_dataframe_with_predictions = generative_qa_utils.split_questions_and_contexts_into_two_columns(dataframe=questions_and_answers_df)
dataset_dataframe_with_predictions

,questions,contexts,labels,predictions
0,How does the code add a user in the given buck...,def add_bucket_default_owner bucket_name user_...,as an owner,by name
1,Does the code add a user in the given buckets ...,def add_bucket_default_owner bucket_name user_...,Yes,Yes
2,Where does the code add a user as an owner?,def add_bucket_default_owner bucket_name user_...,in the given buckets default object access con...,in a bucket
3,Does the code create a simple sdist tarball at...,def make_trivial_sdist dist_path setup_py setu...,Yes,Yes
4,How does the code create a simple sdist tarball?,def make_trivial_sdist dist_path setup_py setu...,at dist_path,using the given setup
...,...,...,...,...
3425,What does the code add to a virtual service?,def add_server protocol None service_address N...,a real server,a server
3426,What does the code get?,def get_codon_alphabet alphabet gap '-' stop '...,alignment alphabet for codon alignment,the codon alphabet
3427,What does the code get?,def get_ancestors x collection None if collect...,ancestor random variables of input,the list of possible candidates for the given ...
3428,What does the code perform?,def lookupNameservers name timeout None return...,an ns record lookup,a record lookup


In [25]:
bleu_result, rogue_result, meteor_result= generative_qa_utils.get_metrics(dataset_dataframe_with_predictions)
print(
    "BLEU:\n",
    bleu_result,
    "\nROGUE:\n",
    rogue_result,
    "\nMETEOR:\n",
    meteor_result
)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BLEU:
 {'bleu': 0.08257298141743828, 'precisions': [0.4183647369767244, 0.1635136763465478, 0.09492726594554271, 0.06425948592411261], 'brevity_penalty': 0.5777349249977212, 'length_ratio': 0.6457278132569918, 'translation_length': 11729, 'reference_length': 18164} 
ROGUE:
 {'rouge1': 0.34269150701559653, 'rouge2': 0.1052504051751496, 'rougeL': 0.33325207286795455, 'rougeLsum': 0.3325781703895115} 
METEOR:
 {'meteor': 0.2233127058663237} 
VERTSCORE:
 {'precision': [0.8356630206108093, 1.000000238418579, 0.8723040819168091, 1.000000238418579, 0.852370023727417, 0.9296540021896362, 0.8875028491020203, 0.9992695450782776, 0.8870806097984314, 1.000000238418579, 0.9390777349472046, 1.000000238418579, 1.0, 0.9097198843955994, 0.9749985337257385, 1.000000238418579, 1.000000238418579, 0.8469901084899902, 0.9992695450782776, 0.8454920649528503, 0.9012614488601685, 1.000000238418579, 0.8713245391845703, 0.9992695450782776, 1.000000238418579, 0.884512722492218, 1.000000238418579, 0.88455307483673

In [16]:
rows_with_closed_questions = dataset_dataframe_with_predictions[
    (dataset_dataframe_with_predictions['labels'] == 'Yes') | 
    (dataset_dataframe_with_predictions['labels'] == 'No')].reset_index(drop=True)

rows_with_closed_questions_correct_answers = rows_with_closed_questions[
    (rows_with_closed_questions['labels'] == rows_with_closed_questions['predictions'])].reset_index(drop=True)

rows_with_closed_questions_wrong_answers = rows_with_closed_questions[
    (rows_with_closed_questions['labels'] != rows_with_closed_questions['predictions'])].reset_index(drop=True)

rows_with_long_answers_for_closed_questions = rows_with_closed_questions[
    (rows_with_closed_questions['labels'] != 'Yes') & 
    (rows_with_closed_questions['labels'] != 'No')].reset_index(drop=True)

In [17]:
number_of_closed_questions = len(rows_with_closed_questions)
number_of_correct_answers = len(rows_with_closed_questions_correct_answers)
number_of_wrong_answers = len(rows_with_closed_questions_wrong_answers)

number_of_long_answers_for_closed_questions = len(rows_with_long_answers_for_closed_questions)

print("Correct answers: ", number_of_correct_answers)
print("Wrong answers: ", number_of_wrong_answers)
print("Long answers for closed questions: ", number_of_long_answers_for_closed_questions)
print("Wrong answers without the long ones: ", number_of_wrong_answers - number_of_long_answers_for_closed_questions)

Correct answers:  116
Wrong answers:  35
Long answers for closed questions:  0
Wrong answers without the long ones:  35


In [26]:
# Save all relevant training and evaluation metrics to a json file.
evaluation_data = {
    "training": {
        "metrics": history.history,
        "attempted_epochs": train_epochs,
        "best_epoch": best_epoch,
        "training_time": time_measure_cb.total_training_time(),
        "gpu": core_qa_utils.get_gpu_name(),
    },
    "test_set": {
        "loss": loaded_model_evaluation,
        "bleu": bleu_result,
        "rogue": rogue_result,
        "meteor": meteor_result,
        "closed_questions": {
            "closed_questions_number": number_of_closed_questions,
            "correct_answers_number": number_of_correct_answers,
            "wrong_answers_number": number_of_wrong_answers,
            "long_answers_for_closed_questions_number": number_of_long_answers_for_closed_questions,
        }
    },
}

core_qa_utils.save_dict_as_json(
    evaluation_data, dir_path=model_evaluation_dir, filename="evaluation_data.json"
)

In [38]:
samples_for_manual_check = 50

predictions_for_manual_check = dataset_dataframe_with_predictions.sample(n = samples_for_manual_check).reset_index(drop=True)
predictions_for_manual_check

,questions,contexts,labels,predictions
0,What does the code setup?,def setup_platform hass config add_devices dis...,the steam platform,the steam sensor
1,What does the code get?,@frappe whitelist def get_opening_accounts com...,all balance sheet accounts for opening entry,the opening accounts for a given company
2,What did trimpoints set for all sff files in d...,def set_sff_trimpoints sff_dir technical_lengt...,to end of technical read,to be clipped
3,For what purpose do class decorator attach the...,def notification_sample sample def wrap cls if...,for documentation generation purposes,to return a notification sample
4,Where does the code get a languages name?,def get_language_name language_code try lang_d...,in the currently active locale,in the given language code
5,What does the code generate?,def make_sampling_table size sampling_factor 1...,a word rank - based probabilistic sampling table,a sampling table
6,What does the code generate?,def generate_secret_key return _get_random_string,a new securely - generated secret key appropri...,a random key
7,What should all the sem compliant tools have?,def generate_all_classes modules_list [] launc...,wrappers created for them,the same name
8,What does the code subtract from the subtractf...,def subtractXIntersectionsTable subtractFromTa...,the subtracttable,the x intersections table
9,What does the code create?,def make_traceback exc_info source_hint None e...,a processed traceback object from the exc_info,a traceback for the given exc_info


In [20]:
# import evaluate

# bleu_metric = evaluate.load("bleu")
# rouge_metric = evaluate.load("rouge")
# meteor_metric = evaluate.load("meteor")
# bertscore_metric = evaluate.load("bertscore")
# sacrebleu_metric = evaluate.load("sacrebleu")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [47]:
dataset_dataframe_with_predictions = dataset_dataframe_with_predictions.sample(n = 100).reset_index(drop=True)
dataset_dataframe_with_predictions

,index,questions,contexts,labels,predictions
0,3103,How is the statement mutating?,def is_mutating status if not status return Fa...,based on the status,according to the given status
1,2974,For what purpose does a queue class wrap if cl...,def track_closed cls class TrackingClosed cls ...,to track down,to track the closed method
2,3260,When is this run?,def plugins_update for env in PluginGlobals en...,when plugins have been loaded or unloaded,currently
3,1259,What does the code get?,@frappe whitelist def get_default_address_temp...,default address template,the default email address template
4,3172,What does the code run?,def _dscl cmd ctype 'create' if __grains__['os...,a dscl -create command,a dll cli example
...,...,...,...,...,...
95,1686,What do signed url provide to a resource?,def generate_signed_url credentials resource e...,query - string authn,the given credentials
96,1676,What does the code find?,def pr_get_ancestors pe_id s3db current s3dbat...,all ancestor entities of a person entity in th...,the ancestors of a pe
97,2689,What does the code return?,def css *args return '\\n' join [stylesheet_li...,appropriate string of link tags,a list of stylesheet links
98,426,What has a fixed prime / generator group?,def isFixedGroup kexAlgorithm return _IFixedGr...,c{kexalgorithm,the kex algorithm


In [48]:
# rogue1_list = []
# rogue2_list = []
# rogueL_list = []

# for index, row in tqdm(dataset_dataframe_with_predictions.iterrows()):
#     rogue1_list.append(rouge_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['rouge1']
#         )
#     rogue2_list.append(rouge_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['rouge2']
#         )
#     rogueL_list.append(rouge_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['rougeL']
#         )
  
# dataset_dataframe_with_predictions['rouge1'] = rogue1_list
# dataset_dataframe_with_predictions['rouge2'] = rogue2_list
# dataset_dataframe_with_predictions['rougeL'] = rogueL_list  

100it [00:35,  2.82it/s]


In [104]:
# bleu_list = []
# bleu1_list = []
# bleu2_list = []

# for index, row in tqdm(dataset_dataframe_with_predictions.iterrows()):
#     bleu_list.append(bleu_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['bleu']
#         )
#     bleu1_list.append(bleu_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['precisions'][0]
#         )
#     bleu2_list.append(bleu_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['precisions'][1]
#         )

# dataset_dataframe_with_predictions['bleu'] = bleu_list
# dataset_dataframe_with_predictions['bleu1'] = bleu1_list
# dataset_dataframe_with_predictions['bleu2'] = bleu2_list

100it [00:01, 69.27it/s]


In [51]:
# meteor_list = []

# for index, row in tqdm(dataset_dataframe_with_predictions.iterrows()):
#     meteor_list.append(meteor_metric.compute(
#         predictions=[row['predictions']], references=[row['labels']])['meteor']
#         )
    
# dataset_dataframe_with_predictions['meteor'] = meteor_list

100it [00:00, 151.54it/s]


In [18]:
dataset_dataframe_with_predictions = generative_qa_utils.calculate_bleus_for_each_sample(dataset_dataframe_with_predictions)
dataset_dataframe_with_predictions = generative_qa_utils.calculate_rouges_for_each_sample(dataset_dataframe_with_predictions)
dataset_dataframe_with_predictions = generative_qa_utils.calculate_meteor_for_each_sample(dataset_dataframe_with_predictions)
dataset_dataframe_with_predictions

3430it [01:08, 50.43it/s]
3430it [30:39,  1.86it/s]
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
3430it [00:36, 93.71it/s] 


,index,questions,contexts,labels,predictions,bleu,bleu1,bleu2,rouge1,rouge2,rougeL,meteor
0,0,How does the code add a user in the given buck...,def add_bucket_default_owner bucket_name user_...,as an owner,by name,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
1,1,Does the code add a user in the given buckets ...,def add_bucket_default_owner bucket_name user_...,Yes,Yes,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.500000
2,2,Where does the code add a user as an owner?,def add_bucket_default_owner bucket_name user_...,in the given buckets default object access con...,in a bucket,0.0,0.333333,0.0,0.166667,0.0,0.166667,0.119048
3,3,Does the code create a simple sdist tarball at...,def make_trivial_sdist dist_path setup_py setu...,Yes,Yes,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.500000
4,4,How does the code create a simple sdist tarball?,def make_trivial_sdist dist_path setup_py setu...,at dist_path,using the given setup,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3425,3425,What does the code add to a virtual service?,def add_server protocol None service_address N...,a real server,a server,0.0,1.000000,0.0,0.800000,0.0,0.800000,0.344828
3426,3426,What does the code get?,def get_codon_alphabet alphabet gap '-' stop '...,alignment alphabet for codon alignment,the codon alphabet,0.0,0.666667,0.0,0.500000,0.0,0.250000,0.208333
3427,3427,What does the code get?,def get_ancestors x collection None if collect...,ancestor random variables of input,the list of possible candidates for the given ...,0.0,0.111111,0.0,0.142857,0.0,0.142857,0.092593
3428,3428,What does the code perform?,def lookupNameservers name timeout None return...,an ns record lookup,a record lookup,0.0,0.666667,0.5,0.571429,0.4,0.571429,0.480769


In [19]:
thresholds = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

In [21]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "rouge1", thresholds)

[1287, 905, 553, 377, 308]

In [22]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "rouge2", thresholds)

[2791, 300, 153, 76, 110]

In [23]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "rougeL", thresholds)

[1336, 928, 522, 340, 304]

In [24]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "bleu", thresholds)

[3340, 23, 17, 18, 32]

In [25]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "bleu1", thresholds)

[1204, 578, 585, 415, 648]

In [26]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "bleu2", thresholds)

[2737, 208, 183, 93, 209]

In [27]:
generative_qa_utils.calculate_prediction_numbers_per_metric_range(dataset_dataframe_with_predictions, "meteor", thresholds)

[2050, 680, 363, 193, 144]

In [ ]:
dataset_dataframe_with_predictions.to_csv(
    model_evaluation_dir / "test_set_sample_generation.csv", index=True, index_label="index"
)

In [5]:
dataset_dataframe_with_predictions = pd.read_csv(model_evaluation_dir / "test_set_sample_generation.csv")
dataset_dataframe_with_predictions

,index,index.1,questions,contexts,labels,predictions,bleu,bleu1,bleu2,rouge1,rouge2,rougeL,meteor
0,0,0,How does the code add a user in the given buck...,def add_bucket_default_owner bucket_name user_...,as an owner,by name,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
1,1,1,Does the code add a user in the given buckets ...,def add_bucket_default_owner bucket_name user_...,Yes,Yes,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.500000
2,2,2,Where does the code add a user as an owner?,def add_bucket_default_owner bucket_name user_...,in the given buckets default object access con...,in a bucket,0.0,0.333333,0.0,0.166667,0.0,0.166667,0.119048
3,3,3,Does the code create a simple sdist tarball at...,def make_trivial_sdist dist_path setup_py setu...,Yes,Yes,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.500000
4,4,4,How does the code create a simple sdist tarball?,def make_trivial_sdist dist_path setup_py setu...,at dist_path,using the given setup,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,3425,3425,What does the code add to a virtual service?,def add_server protocol None service_address N...,a real server,a server,0.0,1.000000,0.0,0.800000,0.0,0.800000,0.344828
3426,3426,3426,What does the code get?,def get_codon_alphabet alphabet gap '-' stop '...,alignment alphabet for codon alignment,the codon alphabet,0.0,0.666667,0.0,0.500000,0.0,0.250000,0.208333
3427,3427,3427,What does the code get?,def get_ancestors x collection None if collect...,ancestor random variables of input,the list of possible candidates for the given ...,0.0,0.111111,0.0,0.142857,0.0,0.142857,0.092593
3428,3428,3428,What does the code perform?,def lookupNameservers name timeout None return...,an ns record lookup,a record lookup,0.0,0.666667,0.5,0.571429,0.4,0.571429,0.480769


In [112]:
questions = dataset_dataframe_with_predictions['questions']
questions = [question.lower() for question in questions]
questions

['how does the code add a user in the given buckets default object access control list?',
 'does the code add a user in the given buckets default object access control list as an owner?',
 'where does the code add a user as an owner?',
 'does the code create a simple sdist tarball at dist_path?',
 'how does the code create a simple sdist tarball?',
 'what does the code create at dist_path?',
 'where does le dirs set?',
 'does le dirs set in parent_dir?',
 'what sets in parent_dir?',
 'does the code get the key for a location in a policy file?',
 'what does the code get?',
 'does the code remove an user from an object like?',
 'how does the code remove an user from an object?',
 'what does the code ensure?',
 'when are all associated messagecategories deleted?',
 'does the code ensure?',
 'does the code run a command on the operation system?',
 'what does the code run?',
 'does none mean?',
 'what does none mean?',
 'what returns along an axis?',
 'does the index of the minimum value re

In [75]:
whats = list(filter(lambda x: x.startswith('what'), questions))
a = len(whats)

In [76]:
whys = list(filter(lambda x: x.startswith('why'), questions))
b = len(whys)

In [77]:
wheres = list(filter(lambda x: x.startswith('where'), questions))
c = len(wheres)

In [78]:
hows = list(filter(lambda x: x.startswith('how'), questions))
d = len(hows)

In [79]:
forwhats = list(filter(lambda x: x.startswith('for what'), questions))
e = len(forwhats)

In [102]:
dos = list(filter(lambda x: x.startswith('do') or x.startswith('does') or x.startswith('did') or x.startswith('will') or x.startswith('is') or x.startswith('are')
                  or x.startswith('ca') or x.startswith('can'), questions))
f = len(dos)

In [115]:
rows_with_closed_questions = dataset_dataframe_with_predictions[
    (dataset_dataframe_with_predictions['labels'] == 'Yes') | 
    (dataset_dataframe_with_predictions['labels'] == 'No')].reset_index(drop=True)
closed = [question.lower() for question in rows_with_closed_questions['questions']]
f = len(closed)

In [81]:
whens = list(filter(lambda x: x.startswith('when'), questions))
g = len(whens)

In [82]:
inwhichs = list(filter(lambda x: x.startswith('in which'), questions))
h = len(inwhichs)

In [107]:
a, b, c, d, e, f, g, h

(2516, 8, 183, 291, 130, 150, 115, 25)

In [116]:
(a + b + c + d + e + f + g + h), len(questions)

(3419, 3430)

In [117]:
used = whats + whys + wheres + hows + forwhats + closed + whens + inwhichs
len(used)

3419

In [118]:
# not_used = list(set(questions).difference(used))
not_used = [item for item in questions if item not in used]
len(not_used)

11

In [119]:
not_used

['by how much did yaml string quote?',
 'by how much does a stub module nt alter system state?',
 'till when do taskqueue tasks execute?',
 'by how much does bitssharp nt need it?',
 'till when is the code puts the process to sleep?',
 'who is asking them?',
 'till when do the calling greenlet wait?',
 'applies the linear operator which organization?',
 'by how much did http response headers encode?',
 'till when do steps perform?',
 'till when do lookup repeat?']

In [29]:
2966+452

3418

In [30]:
seen = set()
repeated = set()
for l in used:
  for i in set(l):
    if i in seen:
      repeated.add(i)
    else:
      seen.add(i)

In [34]:
len(list(repeated))

55